# 🧹 Pandas 데이터 전처리 실습 정리

## 📌 1. 데이터 전처리란?
데이터 전처리(Data Preprocessing)는 **원시 데이터를 분석 가능한 형태로 정리**하는 과정입니다.

### ✅ 전처리의 필요성
- 원시 데이터는 비정형적이고 불완전함
- 결측치(NaN), 이상치(outlier), 중복값 등이 포함됨
- 통일되지 않은 값 존재 (예: ‘남’, ‘남자’ / ‘경북’, ‘경상북도’)
- 분석 및 모델링 정확도 향상을 위해 필수적
- 데이터 전처리 작업은 전체 분석 과정의 70% 이상을 차지

> ⚠️ Garbage In, Garbage Out: 전처리가 부실하면 결과도 신뢰할 수 없음.


## 🧪 2. 자료형 변환 (`astype`)
- `DataFrame.astype(dtype)`: 특정 컬럼의 자료형을 변환
- `errors='ignore'`로 설정 시 변환 실패시 무시
- 예: `df['col1'].astype('float')`

### 관련 함수들:
- `pd.to_datetime()`: 날짜형 변환
- `pd.to_numeric()`: 숫자형 변환
- `convert_dtypes()`: 자동으로 적절한 자료형으로 변환



In [2]:
# 자료형 변환 예시
import pandas as pd

df = pd.DataFrame({
    'col1': ['1', '2', '3'],
    'col2': ['4.5', '5.5', '6.5']
})
df = df.astype({'col1': 'int32', 'col2': 'float'})
print(df.dtypes)
print(df)

col1      int32
col2    float64
dtype: object
   col1  col2
0     1   4.5
1     2   5.5
2     3   6.5


## 🧩 3. GroupBy: 그룹 단위 연산

### 📍 개요
- `Split → Apply → Combine` 패턴으로 동작
- 다양한 통계 연산(합계, 평균, 표준편차 등)을 그룹별로 처리 가능

### 📂 GroupBy의 3가지 핵심 연산

#### ① Aggregation (집계 연산)
- 그룹별로 하나의 **요약값(단일 스칼라)**을 계산하는 연산입니다.

In [3]:
# GroupBy 집계 연산 예시
import pandas as pd

df = pd.DataFrame({
    'A': ['foo', 'foo', 'bar', 'bar', 'foo'],
    'C': [10, 20, 30, 40, 50]
})
result = df.groupby("A")["C"].agg(["sum", "mean", "std"])
print(result)

     sum       mean        std
A                             
bar   70  35.000000   7.071068
foo   80  26.666667  20.816660


#### ② Transformation (변환 연산)
- 그룹별로 데이터를 **변형하되, 결과는 원래와 같은 크기의 DataFrame**으로 유지됩니다.
```python
df["C_mean_diff"] = df.groupby("A")["C"].transform(lambda x: x - x.mean())
```

#### ③ Filtration (필터링 연산)
- 조건에 따라 **특정 그룹 전체를 제거하거나 유지**합니다.
```python
df.groupby("A").filter(lambda x: len(x) >= 2)
```

#### 🛠️ 실습 예시
```python
# Aggregation
df.groupby("group")["score"].mean()

# Transformation
df["normalized"] = df.groupby("group")["score"].transform(lambda x: (x - x.mean()) / x.std())

# Filtration
df.groupby("group").filter(lambda x: x["score"].mean() > 70)
```

In [4]:
# Transformation & Filtration 예시
import pandas as pd

df = pd.DataFrame({
    'group': ['A', 'A', 'B', 'B', 'B'],
    'score': [80, 90, 60, 70, 100]
})

# Transformation
df["normalized"] = df.groupby("group")["score"].transform(lambda x: (x - x.mean()) / x.std())
print("Transformation 결과:")
print(df)

# Filtration
filtered = df.groupby("group").filter(lambda x: x["score"].mean() > 70)
print("\nFiltration 결과:")
print(filtered)

Transformation 결과:
  group  score  normalized
0     A     80   -0.707107
1     A     90    0.707107
2     B     60   -0.800641
3     B     70   -0.320256
4     B    100    1.120897

Filtration 결과:
  group  score  normalized
0     A     80   -0.707107
1     A     90    0.707107
2     B     60   -0.800641
3     B     70   -0.320256
4     B    100    1.120897


## 🔁 4. 데이터 재구조화: `pivot`과 `melt` 자세히 이해하기

### 🧷 `pivot`: 길고 납작한 데이터를 넓게 펼치기 (Long → Wide)

```python
DataFrame.pivot(index=행으로 쓸 컬럼, columns=열로 바꿀 컬럼, values=값으로 쓸 컬럼)
```

```python
df = pd.DataFrame({
    '날짜': ['2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02'],
    '지역': ['서울', '부산', '서울', '부산'],
    'PM10': [70, 80, 90, 100]
})
df.pivot(index='날짜', columns='지역', values='PM10')
```

### 🔄 `melt`: 넓은 데이터를 길고 납작하게 변형 (Wide → Long)

```python
DataFrame.melt(id_vars=고정할 컬럼들, value_vars=변형할 열들, var_name=새 열이름, value_name=값열 이름)
```

```python
wide_df = df.pivot(index='날짜', columns='지역', values='PM10').reset_index()
pd.melt(wide_df, id_vars='날짜', var_name='지역', value_name='PM10')
```

### ✅ 정리
| 함수   | 방향         | 주로 사용하는 상황                           |
|--------|--------------|----------------------------------------------|
| `pivot` | Long → Wide | 시계열이나 그룹별 데이터 비교, 시각화 목적 |
| `melt`  | Wide → Long | 머신러닝 입력용, 분석을 위한 표준 구조화     |


In [5]:
# pivot과 melt 예시
import pandas as pd

df = pd.DataFrame({
    '날짜': ['2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02'],
    '지역': ['서울', '부산', '서울', '부산'],
    'PM10': [70, 80, 90, 100]
})

# pivot
pivoted = df.pivot(index='날짜', columns='지역', values='PM10')
print("pivot 결과:")
print(pivoted)

# melt
wide_df = pivoted.reset_index()
melted = pd.melt(wide_df, id_vars='날짜', var_name='지역', value_name='PM10')
print("\nmelt 결과:")
print(melted)

pivot 결과:
지역           부산  서울
날짜                 
2024-01-01   80  70
2024-01-02  100  90

melt 결과:
           날짜  지역  PM10
0  2024-01-01  부산    80
1  2024-01-02  부산   100
2  2024-01-01  서울    70
3  2024-01-02  서울    90
